In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

In [0]:
%cd "/content/drive/My Drive/data/audio"

/content/drive/My Drive/data/audio


In [0]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import glob
import re
import math
import keras
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from random import shuffle
from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
songs = (re.search('.+?(?=\.wav)', x).group(0)
              for x in glob.iglob('*.wav')) 

#build spectrograms for conv net
spectrograms = []
for song in songs:
  sample_rate, samples = wavfile.read(song+'.wav')
  frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
  spectrogram = spectrogram[:,10000:10500] #sample middle 2.5s of audio 
  #np.place(spectrogram, spectrogram == 0, 10**-10) #avoid div by 0 for log
  with open('../annotations/meter/' + song + '.meter', 'r') as file:
      meter = file.readline()

  spectrograms.append([song, spectrogram, meter])


In [0]:
import random
def load_data(spec_data, frac):
    #split = math.ceil(len(spec_data) * frac)
    split = 80
    
    random.shuffle(spec_data)
    
    x_train = np.array([x[1] for x in spec_data[:split]])
    y_train = np.array([x[2] for x in spec_data[:split]])
    
    x_test = np.array([x[1] for x in spec_data[split:]])
    y_test = np.array([x[2] for x in spec_data[split:]])
    
    return (x_train, y_train), (x_test, y_test)
 





In [0]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

(x_train, y_train), (x_test, y_test) = load_data(spectrograms, 0.7) #70% train-test split

#height and width of img
img_x, img_y = x_train.shape[1:]

#reshape x_train, x_test to 4D tensor - (samples, size1,size2,channels)
x_train = x_train.reshape((x_train.shape[0], img_x, img_y, 1))
x_test = x_test.reshape((x_test.shape[0], img_x, img_y, 1))

import cv2
x_train = cv2.merge((x_train,x_train,x_train)).reshape((x_train.shape[0], img_x, img_y, 3))
x_test = cv2.merge((x_test, x_test, x_test)).reshape((x_test.shape[0], img_x, img_y, 3))


#x_train = x_train.reshape((80, 129, 500, 1))
#x_train = cv2.merge((x_train, x_train, x_train))


#Does not work, why?
#stack greyscale img depth-wise to create img in rgb
#x_train = np.dstack((x_train, x_train, x_train))
#x_test = np.dstack((x_test, x_test, x_test))



#process input for compatibility with InceptionV3 model
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 4)
y_test = keras.utils.to_categorical(y_test, 4)


#reshape y_train, y_test to (N,1) tensor for generator
#y_train = y_train.reshape(y_train.shape[0], 1)
#y_test = y_test.reshape(y_test.shape[0], 1)


print(np.sum(y_train, axis = 0)/len(y_train))
print(np.sum(y_test, axis = 0)/len(y_test))




[0.3    0.2875 0.2375 0.175 ]
[0.15789473 0.13157895 0.28947368 0.42105263]


In [0]:
#generator code for model.fit_generator()
def generator(features, labels, batch_size):
  # Create empty arrays to contain batch of features and labels

  batch_features = np.zeros((batch_size, img_x, img_y, 3))
  batch_labels = np.zeros((batch_size,1))

  while True:
   for i in range(batch_size):
     # choose random index in features
     index= np.random.choice(len(x_train))
     batch_features[i] = features[index]
     batch_labels[i] = labels[index]
   yield batch_features, keras.utils.to_categorical(batch_labels, num_classes=4)

In [0]:
q 



Train on 80 samples, validate on 38 samples
Epoch 1/200
80/80 [==============================] - 34s 426ms/step - loss: 1.4603 - acc: 0.2625 - val_loss: 1.4682 - val_acc: 0.1842
Epoch 2/200
80/80 [==============================] - 4s 53ms/step - loss: 1.4050 - acc: 0.2625 - val_loss: 1.5100 - val_acc: 0.2368
Epoch 3/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3517 - acc: 0.3500 - val_loss: 1.4930 - val_acc: 0.2368
Epoch 4/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3640 - acc: 0.3125 - val_loss: 1.4737 - val_acc: 0.2368
Epoch 5/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3251 - acc: 0.3750 - val_loss: 1.4954 - val_acc: 0.1316
Epoch 6/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3484 - acc: 0.2625 - val_loss: 1.4711 - val_acc: 0.2105
Epoch 7/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3009 - acc: 0.4000 - val_loss: 1.4967 - val_acc: 0.2368
Epoch 8/200
80/80 [===========

In [0]:
y_prob = model.predict(x_train) 
y_classes = y_prob.argmax(axis=-1)
y_classes


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [0]:
y_train.reshape(len(y_train),)

array([2, 0, 1, 0, 1, 1, 1, 1, 3, 2, 3, 2, 3, 2, 1, 2, 3, 0, 0, 1, 0, 3,
       3, 3, 0, 3, 0, 3, 2, 3, 3, 1, 2, 1, 2, 3, 2, 0, 3, 2, 1, 3, 2, 2,
       3, 3, 2, 1, 1, 3, 2, 3, 0, 3, 0, 3, 1, 1, 2, 0, 3, 3, 3, 2, 0, 0,
       1, 3, 1, 0, 2, 1, 3, 2, 2, 0, 0, 1, 2, 2])

In [0]:

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(generator(x_train, y_train, 5), steps_per_epoch=16, epochs=20)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',
              metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(generator(x_train, y_train, 5), steps_per_epoch=16, epochs=20)

In [0]:



#score = model.evaluate(x_test, keras.utils.to_categorical(y_test, num_classes=4), verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

Test loss: 13.99729397422389
Test accuracy: 0.13157894736842105


In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
le.transform(y_train).shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(80,)

In [0]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [0]:
a = [1,2,3,4,5]
random.shuffle(a)
a

[3, 1, 2, 4, 5]

In [0]:
drive.unmount('/content/drive/')

AttributeError: ignored

In [0]:
!cat /proc/meminfo
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

MemTotal:       13335212 kB
MemFree:        11191004 kB
MemAvailable:   12657464 kB
Buffers:           44948 kB
Cached:          1596884 kB
SwapCached:            0 kB
Active:           514624 kB
Inactive:        1406360 kB
Active(anon):     257460 kB
Inactive(anon):      328 kB
Active(file):     257164 kB
Inactive(file):  1406032 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               232 kB
Writeback:             0 kB
AnonPages:        279132 kB
Mapped:           154400 kB
Shmem:               844 kB
Slab:             119596 kB
SReclaimable:      89848 kB
SUnreclaim:        29748 kB
KernelStack:        3468 kB
PageTables:         4416 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667604 kB
Committed_AS:    1675276 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12722042531394066010, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3389415551423531816
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8339090031054887459
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13549150098634449447
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

#input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

#model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(img_x, img_y, 3))  

# create the base pre-trained model
base_model = InceptionV3( weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 3s 0us/step


In [0]:
input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

96116736/96112376 [==============================] - 4s 0us/step


In [0]:
model.input

<tf.Tensor 'input_2:0' shape=(?, 224, 224, 3) dtype=float32>

In [0]:
spectrograms[0]


['02_12003_2-02_Pakkala_Nilabadi',
 array([[1.83138561e+01, 5.14330673e+00, 2.04259014e+01, ...,
         1.17135666e+02, 1.27727928e+02, 1.67983398e+01],
        [1.90238983e+02, 1.69976208e+03, 3.05827734e+03, ...,
         9.73505005e+02, 1.40374890e+03, 3.33984888e+03],
        [1.89349350e+02, 2.73411279e+03, 1.44540820e+03, ...,
         5.16861670e+03, 3.34138281e+03, 5.96172998e+03],
        ...,
        [3.30927001e-08, 8.70707299e-06, 4.53788607e-06, ...,
         4.01475472e-06, 4.88131218e-06, 4.22224548e-06],
        [4.80746121e-06, 1.98490488e-06, 2.52803766e-05, ...,
         8.97796872e-06, 1.24984990e-06, 2.03112941e-05],
        [1.07506629e-07, 1.28157330e-06, 8.33216109e-06, ...,
         6.85883642e-13, 1.70183739e-06, 7.59690238e-06]], dtype=float32),
 '7/8']

In [0]:
x_train[0]

array([[[  4.7833514 ,   4.7833514 ,   4.7833514 ],
        [ -0.21646124,  -0.21646124,  -0.21646124],
        [  4.0584393 ,   4.0584393 ,   4.0584393 ],
        ...,
        [  1.1657901 ,   1.1657901 ,   1.1657901 ],
        [  1.6616268 ,   1.6616268 ,   1.6616268 ],
        [ -0.9548845 ,  -0.9548845 ,  -0.9548845 ]],

       [[133.93564   , 133.93564   , 133.93564   ],
        [ 50.45263   ,  50.45263   ,  50.45263   ],
        [101.830086  , 101.830086  , 101.830086  ],
        ...,
        [ 69.55197   ,  69.55197   ,  69.55197   ],
        [107.47329   , 107.47329   , 107.47329   ],
        [ 30.32808   ,  30.32808   ,  30.32808   ]],

       [[ 58.485153  ,  58.485153  ,  58.485153  ],
        [  3.8016324 ,   3.8016324 ,   3.8016324 ],
        [ 67.432205  ,  67.432205  ,  67.432205  ],
        ...,
        [  4.039701  ,   4.039701  ,   4.039701  ],
        [ 90.98996   ,  90.98996   ,  90.98996   ],
        [ 26.159327  ,  26.159327  ,  26.159327  ]],

       ...,

      